In [1]:
# NLI(Natural Language Interference:자연어 추론) 테스트 예제
#
# => input_ids : [CLS]senetence1(전제)[SEP]sentence2(가설)
# => attention_mask : 1111111111(전체,가설)0000000(그외)
# => token_type_ids : 0000000(전제)1111111(가설)00000000(그외)
# => laels : 참(수반:entailment), 거짓(모순:contradiction), 모름(중립:neutral)

import numpy as np
import pandas as pd
import torch
import os
import torch.nn.functional as F
import sys

from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from tqdm.notebook import tqdm

sys.path.append("../../")
from myutils import seed_everything, GPU_info, mlogging


logger = mlogging(loggername="distilbertnlitest", logfilename="distilbertnlitest")
device = GPU_info()
seed_everything(111)

logfilepath:bwdataset_2022-03-31.log
logfilepath:qnadataset_2022-03-31.log
logfilepath:distilbertnlitest_2022-03-31.log
True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30


In [2]:
#############################################################################################
# 변수들 설정
# - model_path : from_pretrained() 로 호출하는 경우에는 모델파일이 있는 폴더 경로나 
#          huggingface에 등록된 모델명(예:'bert-base-multilingual-cased')
#          torch.load(model)로 로딩하는 경우에는 모델 파일 풀 경로
#
# - vocab_path : from_pretrained() 호출하는 경우에는 모델파일이 있는 폴더 경로나
#          huggingface에 등록된 모델명(예:'bert-base-multilingual-cased')   
#          BertTokenizer() 로 호출하는 경우에는 vocab.txt 파일 풀 경로,
#############################################################################################

model_path = '../../model/distilbert/distilbert-0331-TS-nli-0.1-10/24544'
vocab_path = '../../model/distilbert/distilbert-0331-TS-nli-0.1-10/24544'

# tokeniaer 및 model 설정
#tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
tokenizer = DistilBertTokenizer.from_pretrained(vocab_path, do_lower_case=False)
# strip_accents=False : True로 하면, 가자 => ㄱ ㅏ ㅈ ㅏ 식으로 토큰화 되어 버림(*따라서 한국어에서는 반드시 False)
# do_lower_case=False : # 소문자 입력 사용 안함(한국어에서는 반드시 False)
#tokenizer = BertTokenizer(vocab_file=vocab_path, strip_accents=False, do_lower_case=False) 
                       
# 레벨을 1개만 선택하는 경우
model = DistilBertForSequenceClassification.from_pretrained(model_path, num_labels=3)
#model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=6)

# 레벨을 멀티로 선택해야 하는 경우
#model = BertForSequenceClassification.from_pretrained(model_path, problem_type="multi_label_classification",num_labels=6)

model.to(device)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DistilBertTokenizer'.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(167550, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
      

In [3]:
# tokenier 테스트
print(len(tokenizer))
print(tokenizer.encode("눈에 보이는 반전이었지만 영화의 흡인력은 사라지지 않았다", "정말 재미있다"))
print(tokenizer.convert_ids_to_tokens(131027))
print(tokenizer.convert_tokens_to_ids('정말'))

167550
[101, 9034, 10530, 124997, 11018, 125215, 10739, 69708, 42428, 10459, 10020, 129937, 10892, 132489, 12508, 49137, 102, 9670, 89523, 125551, 76820, 102]
최치원
100


In [4]:
# 평가 data loader 생성

from torch.utils.data import DataLoader, RandomSampler

sys.path.append("..")
from myutils import ClassificationDataset, KlueNLICorpus, data_collator, KorNLICorpus
#############################################################################
# 변수 설정
#############################################################################
max_seq_len = 128   # 글자 최대 토큰 길이 해당 토큰 길이 이상은 잘린다.
batch_size = 32        # 배치 사이즈(64면 GUP Memory 오류 나므로, 32 이하로 설정할것=>max_seq_length 를 줄이면, 64도 가능함)
cache = True   # 캐쉬파일 생성할거면 True로 (True이면 loding할때 캐쉬파일있어도 이용안함)
#############################################################################

# corpus 파일 설정
#corpus = KlueNLICorpus()
corpus = KorNLICorpus()

# 평가 dataset 생성
#file_fpath = '../../korpora/klue-nli/klue-nli-v1.1_dev.json'
file_fpath = '../../korpora/kornli/xnli.test.ko.tsv'
    
dataset = ClassificationDataset(file_fpath=file_fpath, max_seq_length=max_seq_len, tokenizer=tokenizer, corpus=corpus, overwrite_cache=cache)

# 평가 dataloader 생성
eval_loader = DataLoader(dataset, 
                          batch_size=batch_size, 
                          #shuffle=True, # dataset을 섞음
                          sampler=RandomSampler(dataset, replacement=False), #dataset을 랜덤하게 샘플링함
                          collate_fn=data_collator, # dataset을 tensor로 변환(예시 {'input_ids':tensor[0,1,2,3,1,], 'token_type_id:tensor[0,0,0,0,0], 'attention_mask:tensor[1,1,1,1,1], 'labels':tensor[5]}
                          num_workers=4)

print('eval_loader_len: {}'.format(len(eval_loader)))

2022-03-31 20:06:27,161 - bwpdataset - INFO - Creating features from dataset file at ../../korpora/kornli/xnli.test.ko.tsv
2022-03-31 20:06:27,163 - bwpdataset - INFO - loading data... LOOKING AT ../../korpora/kornli/xnli.test.ko.tsv
2022-03-31 20:06:27,225 - bwpdataset - INFO - tokenize sentences, it could take a lot of time...
2022-03-31 20:06:28,893 - bwpdataset - INFO - tokenize sentences [took 1.667 s]


  0%|          | 0/5010 [00:00<?, ?it/s]

2022-03-31 20:06:28,924 - bwpdataset - INFO - *** Example ***
2022-03-31 20:06:28,925 - bwpdataset - INFO - sentence A, B: 글쎄, 나는 그것에 관해 생각조차 하지 않았지만, 나는 너무 좌절했고, 결국 그에게 다시 이야기하게 되었다. + 나는 그와 다시 이야기하지 않았다.
2022-03-31 20:06:28,927 - bwpdataset - INFO - tokens: [CLS] [UNK] , 나는 그것 ##에 관 ##해 생각 ##조 ##차 하지 않 ##았 ##지만 , 나는 너 ##무 좌절 ##했고 , 결국 그 ##에게 다시 이야기 ##하게 되었다 . [SEP] 나는 그 ##와 다시 이야기 ##하지 않았다 . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
2022-03-31 20:06:28,927 - bwpdataset - INFO - label: contradiction
20

eval_loader_len: 157


In [5]:
import time

logger.info(f"=== model: {model_path} ===")
logger.info(f"eval_file : {file_fpath}")
logger.info(f"num_parameters: {model.num_parameters()}")

# 평가 시작
model.eval()

total_loss = 0
total_len = 0
total_correct = 0

start = time.time()
logger.info(f'---------------------------------------------------------')

for data in tqdm(eval_loader):
    # 입력 값 설정
    # =>**distilbert에는 token_type_ids가 없다
    labels = data['labels'].to(device)
    input_ids = data['input_ids'].to(device)
    attention_mask = data['attention_mask'].to(device)
 
    # 손실률 계산하는 부분은 no_grade 시켜서, 계산량을 줄임.
    # => torch.no_grad()는 gradient을 계산하는 autograd engine를 비활성화 하여 
    # 필요한 메모리를 줄이고, 연산속도를 증가시키는 역활을 함
    with torch.no_grad():
        # 모델 실행
        outputs = model(input_ids=input_ids, 
                       attention_mask=attention_mask,
                       labels=labels)
    
        # 출력값 loss,logits를 outputs에서 얻어옴
        loss = outputs.loss
        logits = outputs.logits

        pred = torch.argmax(F.softmax(logits), dim=1)
        correct = pred.eq(labels)
        total_correct += correct.sum().item()
        total_len += len(labels)

logger.info(f"eval-accuracy: {total_correct / total_len}")
logger.info(f'---------------------------------------------------------')
logger.info(f'=== 처리시간: {time.time() - start:.3f} 초 ===')
logger.info(f'-END-\n')

2022-03-31 20:06:29,287 - distilbertnlitest - INFO - === model: ../../model/distilbert/distilbert-0331-TS-nli-0.1-10/24544 ===
2022-03-31 20:06:29,289 - distilbertnlitest - INFO - eval_file : ../../korpora/kornli/xnli.test.ko.tsv
2022-03-31 20:06:29,290 - distilbertnlitest - INFO - num_parameters: 172193283
2022-03-31 20:06:29,292 - distilbertnlitest - INFO - ---------------------------------------------------------


  0%|          | 0/157 [00:00<?, ?it/s]

/tmp/ipykernel_136123/247420279.py:37: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred = torch.argmax(F.softmax(logits), dim=1)
2022-03-31 20:06:38,014 - distilbertnlitest - INFO - eval-accuracy: 0.7163672654690619
2022-03-31 20:06:38,017 - distilbertnlitest - INFO - ---------------------------------------------------------
2022-03-31 20:06:38,019 - distilbertnlitest - INFO - === 처리시간: 8.727 초 ===
2022-03-31 20:06:38,020 - distilbertnlitest - INFO - -END-

